In [1]:
from loader import load_reviews_to_list
from text_search import *

In [2]:
path = 'Data\\Restaurants_London_England.tsv'
reviews = load_reviews_to_list(path)

In [3]:
tags, bags = generate_inverted_index(reviews)

In [4]:
def print_results(results_index_list, reviews, exec_time):
    for i in reviews_index:
        print(reviews[i])

In [10]:
q = ['italian', 'reservations', 'late night']
result_indexes_raw, t_raw = kwSearchRaw(q, reviews)

result_indexes_if, t_if = kwSearchIF(q, tags, bags)

In [11]:
len(result_indexes_raw), t_raw

(260, 0.004984378814697266)

In [12]:
len(result_indexes_if), t_if

(260, 0.0009999275207519531)

In [8]:
#bags, tags = zip(*sorted(zip(bags,tags), key=lambda b: len(b[0])))